In [6]:
import time
import random
import numpy as np
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.document_loaders import CSVLoader
from IPython.display import display, Markdown

import langchain
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.llms import OpenAI
from langchain.evaluation.qa import QAGenerateChain
from langchain.evaluation.qa import QAEvalChain
from langchain.text_splitter import TokenTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma

from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from uuid import uuid4

import openai
import os
from dotenv import load_dotenv
import getpass
import logging
import pandas as pd
import json

# Configurar o logging para salvar a saída de debug em um arquivo
logging.basicConfig(
    filename='./logs/debug_output.log',  # O arquivo onde os logs serão salvos
    level=logging.DEBUG,          # O nível de log (DEBUG para capturar tudo)
    format='%(asctime)s - %(levelname)s - %(message)s'  # Formato do log
)

logger = logging.getLogger()

import warnings
warnings.filterwarnings("ignore")


In [7]:
load_dotenv() # read local .env file

if not os.getenv("OPENAI_API_KEY"):
   os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [107]:
file = 'inputs/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding="utf-8")
docs = loader.load()

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=10)

split_documents = []
for doc in docs:
    split_docs = text_splitter.split_documents([doc])
    split_documents.extend(split_docs)
    
    
documents_dict = [
    {"page_content": doc.page_content, "metadata": doc.metadata} for doc in split_documents
]
# Save the list of dictionaries to a JSON file
with open("inputs/documents_split_langchain.json", "w") as file:
    json.dump(documents_dict, file, indent=4)

logger.debug("Documents have been saved to 'documents_split_langchain.json'")

In [103]:
# Load documents split
with open("inputs/documents_split_langchain.json", "r") as file:
    documents_dict = json.load(file)

# Convert the list of dictionaries back to a list of Document objects
documents = [
    Document(page_content=doc["page_content"], metadata=doc["metadata"])
    for doc in documents_dict
]

In [12]:
from functions import openai_embedding_function, hf_embeddings_function, google_embedding_function 
# Generating embeddings 
openai_embedding_function(documents, model = "text-embedding-3-large")

In [13]:
from functions import load_VectorStore

In [42]:
embeddings_provider = 'OpenAI'
vector_store = load_VectorStore(embeddings_provider)

OpenAI db loaded


## Criando as querys para validar os metodos

In [149]:
langchain_debug = False

manual_examples = [
     {"query": "Do the Cozy Comfort Pullover Set have side pockets?", 
      "ground_truths": "Yes"},
    
     {"query": "What collection is the Ultra-Lofty \850 Stretch Down Hooded Jacket from?",
      "ground_truths": "The DownTek collection"
     }]

 # Gerando exemplos para validação com IA generativa
 # Gerando examples via LLM 

llm_model = "gpt-4o-mini"
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

select_n_documents = 5

# Selecionar 5 documentos aleatórios da lista `docs`
random_documents = random.sample(docs, select_n_documents)

# Criando a estrutura de entrada (lista de dicionários)
formatted_documents = [{"doc": doc} for doc in random_documents]

# Aplicando o processo de exemplo genérico (example_gen_chain) e obtendo a saída
gen_examples = example_gen_chain.apply_and_parse(formatted_documents)

# Ajustando a saída para incluir o contexto
gen_examples_adjusted_examples = []
for doc, example in zip(random_documents, gen_examples):
    qa_pair = example.get('qa_pairs', {})
    query = qa_pair.get('query', '')
    answer = qa_pair.get('answer', '')
    
    gen_examples_adjusted_examples.append({
        'context': doc,  # Adicionando o documento original como contexto
        'query': query,
        'ground_truths': answer
    })

validation_examples = gen_examples_adjusted_examples

documents_dict = [
    [{"context": doc['context'].page_content, 
      "metadata": doc['context'].metadata,
      "ground_truths": doc['ground_truths'], 
      "query": doc['query']}   
     for doc in data]
    for data in [validation_examples]
]
documents_dict

os.makedirs('results', exist_ok=True)

# Save the list of dictionaries to a JSON file
with open(f"results/query_{embeddings_provider}_results.json", "w") as file:
    json.dump(documents_dict, file, indent=4)

print(f"Documents have been saved to 'query_{embeddings_provider}_results.json'")

[chain/start] [chain:QAGenerateChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:QAGenerateChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\npage_content=': 514\nname: Women's Trail Model 4 All-Weather Hiking Shoes\ndescription: Supercomfortable lightweight hikers with a built-in waterproof membrane.\r\n\r\nSpecs: Approx. weight: 1 lb. 13 oz.\r\n\r\nConstruction: 's exclusive VertiGrip outsole provides excellent traction on a variety of surfaces. Suede-and-fabric upper with a waterproof TEK2.5® barrier keeps feet dry. Cushioned EVA midsole and removable footbed pro

# Comparando métodos de RAG

In [ ]:
def baseline_similarity_method(validation_examples, vector_store, embeddings_provider, langchain_debug=False):
    
    model = 'gpt-3.5-turbo-instruct'

    # Initialize the language model
    llm = OpenAI(temperature=0.2, model=model)

    # Set up the retriever with similarity search
    retriever = vector_store.as_retriever(search_type="similarity", k=1)  # Retrieve the most relevant chunk

    # Initialize the RetrievalQA chain
    qa = RetrievalQA.from_chain_type(
        llm=llm, 
        chain_type="stuff",
        retriever=retriever, 
        verbose=True,
        chain_type_kwargs={"document_separator": "\n"}
    )

    # Log debug information if debug is enabled
    # if langchain_debug:
    #    langchain_debug = True

    # Apply the model to the validation examples
    predictions = qa.apply(validation_examples)

    # Prepare the results
    baselinemethod_documents_dict = [
        [{"context": doc['context'].page_content, 
          "metadata": doc['context'].metadata,
          "query": doc['query'],
          "ground_truths": doc['ground_truths'],  
          "answer": doc['result']}
         for doc in data]
        for data in [predictions]
    ]

    # Ensure the results directory exists
    os.makedirs('results', exist_ok=True)

    # Save the results to a JSON file
    results_filename = f"results/query_{embeddings_provider}_baselinemethod_documents_dict_results.json"
    with open(results_filename, "w") as file:
        json.dump(baselinemethod_documents_dict, file, indent=4)

    print(f"Documents have been saved to '{results_filename}'")

    return baselinemethod_documents_dict

baseline_similarity_method(validation_examples, vector_store, embeddings_provider, langchain_debug=False)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What are the key features and specifications of the Women's Trail Model 4 All-Weather Hiking Shoes as described in the document?",
  "context": ": 514\nname: Women's Trail Model 4 All-Weather Hiking Shoes\ndescription: Supercomfortable lightweight hikers with a built-in waterproof membrane.\r\n\r\nSpecs: Approx. weight: 1 lb. 13 oz.\r\n\r\nConstruction: 's exclusive VertiGrip outsole provides excellent traction on a variety of surfaces. Suede-and-fabric upper with a waterproof TEK2.5® barrier keeps feet dry. Cushioned EVA midsole and removable footbed provide noticeable comfort right out of the box. Heel-and-toe bumpers add durability.\r\n\r\nAdditional Features: Versatile enough for casu

[[{'context': ": 514\nname: Women's Trail Model 4 All-Weather Hiking Shoes\ndescription: Supercomfortable lightweight hikers with a built-in waterproof membrane.\r\n\r\nSpecs: Approx. weight: 1 lb. 13 oz.\r\n\r\nConstruction: 's exclusive VertiGrip outsole provides excellent traction on a variety of surfaces. Suede-and-fabric upper with a waterproof TEK2.5® barrier keeps feet dry. Cushioned EVA midsole and removable footbed provide noticeable comfort right out of the box. Heel-and-toe bumpers add durability.\r\n\r\nAdditional Features: Versatile enough for casual wear and a wide variety of outdoor adventures. Imported.\r\n\r\nQuestions? Please contact us for more information.",
   'metadata': {'source': 'inputs/OutdoorClothingCatalog_1000.csv',
    'row': 514},
   'query': "What are the key features and specifications of the Women's Trail Model 4 All-Weather Hiking Shoes as described in the document?",
   'ground_truths': "The Women's Trail Model 4 All-Weather Hiking Shoes are supercom

In [152]:
# Técnica simples por similaridade

model = 'gpt-3.5-turbo-instruct'

llm = OpenAI(temperature=0.2, model=model)

retriever = vector_store.as_retriever(search_type="similarity", k=1)  # Recupera os 3 chunks mais relevantes

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",
    retriever=retriever, 
    verbose=True,
    chain_type_kwargs = {"document_separator": "\n"})


# Supondo que você queira registrar a variável `langchain.debug`
# langchain_debug = True  # Exemplo de variável para controlar o modo debug

# Log de debug com interpolação correta
logger.debug(f"qa_baselineMethod predictions debug {langchain_debug}")

predictions = qa.apply(validation_examples)

baselinemethod_documents_dict = [
    [{"context": doc['context'].page_content, 
      "metadata": doc['context'].metadata,
       "query": doc['query'],
       "ground_truths": doc['ground_truths'],  
       "answer": doc['result'],
      } 
     for doc in data]
    for data in [predictions]
]


os.makedirs('results', exist_ok=True)

# Save the list of dictionaries to a JSON file
with open(f"results/query_{embeddings_provider}_baselinemethod_documents_dict_results.json", "w") as file:
    json.dump(baselinemethod_documents_dict, file, indent=4)

print(f"Documents have been saved to 'query_{embeddings_provider}_baselinemethod_documents_dict_results.json'")

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What are the key features and specifications of the Women's Trail Model 4 All-Weather Hiking Shoes as described in the document?",
  "context": ": 514\nname: Women's Trail Model 4 All-Weather Hiking Shoes\ndescription: Supercomfortable lightweight hikers with a built-in waterproof membrane.\r\n\r\nSpecs: Approx. weight: 1 lb. 13 oz.\r\n\r\nConstruction: 's exclusive VertiGrip outsole provides excellent traction on a variety of surfaces. Suede-and-fabric upper with a waterproof TEK2.5® barrier keeps feet dry. Cushioned EVA midsole and removable footbed provide noticeable comfort right out of the box. Heel-and-toe bumpers add durability.\r\n\r\nAdditional Features: Versatile enough for casu

# Técnica por similaridade apenas para os documentos mais relevantes 


In [ ]:
def hierarchical_retrieval_for_multiple_queries(vector_store, validation_examples):
    # Define the high-level query template for category search
    category_prompt = """
    You are an expert in outdoor clothing products. Based on the following query, 
    return a list of relevant categories (e.g., jackets, pants) from the catalog.
    Query: {query}
    """

    category_template = PromptTemplate(input_variables=["query"], template=category_prompt)
    category_chain = LLMChain(prompt=category_template, llm=OpenAI())

    # Define the refined search for documents within each category
    document_prompt = """
    You are an assistant for question-answering tasks about the products from a store that sells outdoor clothing. 
    Your function is to use the catalog of retrieved context to answer the client's questions based on the following query and the category {category}.
    Query: {query}
    Category: {category}
    Catalog: {catalog}
    """

    document_template = PromptTemplate(input_variables=["query", "category",'catalog'], template=document_prompt)
    document_chain = LLMChain(prompt=document_template, llm=OpenAI())

    for index, item in enumerate(validation_examples):
        query = item['query']

        # Step 2: Use the category to modify the query for document search
        modified_query = f"{query} Category: {category_result}"  # Combine query and category for search
        catalog = vector_store.similarity_search(modified_query, k=5) 

        category_result = category_chain.run({"query": query})
        refined_answer = document_chain.run({"query": query, "category": category_result, "catalog": catalog})
        validation_examples[index]['result'] = refined_answer

    hierarchicalmethod_documents_dict = [
        [{"context": doc['context'].page_content, 
        "metadata": doc['context'].metadata,
        "query": doc['query'],
        "ground_truths": doc['ground_truths'],  
        "answer": doc['result'],
        } 
        for doc in data]
        for data in [predictions]
    ]

    os.makedirs('results', exist_ok=True)

    # Save the list of dictionaries to a JSON file
    with open(f"results/query_{embeddings_provider}_baselinemethod_documents_dict_results.json", "w") as file:
        json.dump(hierarchicalmethod_documents_dict, file, indent=4)

    print(f"Documents have been saved to 'query_{embeddings_provider}_hierarchicalmethod_documents_dict.json'")

    return hierarchicalmethod_documents_dict


# Assuming vector_store is defined as per the previous setup
answers = hierarchical_retrieval_for_multiple_queries(validation_examples, vector_store)
answers


[chain/start] [chain:LLMChain] Entering Chain run with input:
{
  "query": "What are the key features and specifications of the Women's Trail Model 4 All-Weather Hiking Shoes as described in the document?"
}
[llm/start] [chain:LLMChain > llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an expert in outdoor clothing products. Based on the following query, \n    return a list of relevant categories (e.g., jackets, pants) from the catalog.\n    Query: What are the key features and specifications of the Women's Trail Model 4 All-Weather Hiking Shoes as described in the document?"
  ]
}
[llm/end] [chain:LLMChain > llm:OpenAI] [1.28s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n1. Hiking shoes \n2. Women's footwear \n3. Trail Model 4 All-Weather \n4. All-weather shoes \n5. Outdoor shoes \n6. Women's hiking gear \n7. Waterproof shoes \n8. Durable footwear \n9. Trail-ready shoes \n10. Comfortable hiking shoes",
        "generation_in

[{'query': "What are the key features and specifications of the Women's Trail Model 4 All-Weather Hiking Shoes as described in the document?",
  'answer': "\nBased on the query and category provided, the key features and specifications of the Women's Trail Model 4 All-Weather Hiking Shoes can be described as follows:\n\n1. Hiking shoes: These shoes are specifically designed for hiking, with features such as sturdy soles, good grip, and durability.\n\n2. Women's footwear: The shoes are designed for women, with a fit and design that is suitable for the female foot.\n\n3. Trail Model 4 All-Weather: These shoes are part of the Trail Model series, which is known for its durability and all-weather performance.\n\n4. All-weather shoes: The Women's Trail Model 4 All-Weather Hiking Shoes are suitable for all types of weather conditions, making them a versatile choice for outdoor activities.\n\n5. Outdoor shoes: These shoes are designed for outdoor use, with features that make them suitable for 

Define a Retrieval Strategy (Hierarchical Retrieval)
Hierarchical retrieval means you will first retrieve high-level categories or subtopics and then refine the search to get more specific documents.

Step 1: Retrieve relevant high-level categories (e.g., product types like "jackets", "pants", etc.)
Step 2: Within each category, retrieve specific documents (e.g., details of specific jackets or pants).
To achieve this, you may use a two-step retrieval process. First, search for broad categories in the vector store, then use the results to refine the search.
    

In [77]:
# Método Hierarchical Retrieval: Busca Estruturada
def hierarchical_retrieval(query, embedding, top_n=3):
    # Categorias hipotéticas
    categories = {
        "Footwear": ["Women's Campside Oxfords", "Running Shoes"],
        "Dog Mats": ["Recycled Waterhog Dog Mat", "Outdoor Dog Mat"],
        "Swimsuits": ["Infant and Toddler Girls' Coastal Chill Swimsuit"]
    }
    
    # Etapa 1: Identificar a categoria relevante com base na consulta
    category_found = None
    if "dog mat" in query.lower():
        category_found = "Dog Mats"
    elif "shoe" in query.lower():
        category_found = "Footwear"
    elif "swimsuit" in query.lower():
        category_found = "Swimsuits"
    
    if not category_found:
        return "Category not found"

    # Etapa 2: Buscar documentos na categoria encontrada
    category_docs = [doc for doc in documents_dict if category_found in doc["page_content"]]
    
    # Etapa 3: Gerar embeddings para os documentos da categoria
    query_embedding = embedding.embed(query)
    category_embeddings = [embedding.embed(doc["page_content"]) for doc in category_docs]
    
    # Calcular similaridade entre o query e os documentos da categoria
    similarities = [
        (i, embedding.similarity(query_embedding, doc_emb)) for i, doc_emb in enumerate(category_embeddings)
    ]
    
    # Ordenar por similaridade e pegar os top N
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_results = [category_docs[i] for i, _ in similarities[:top_n]]
    
    return top_results

# Exemplo de consulta
query = "Tell me about the dog mat"
results_hierarchical = hierarchical_retrieval(query)

for result in results_hierarchical:
    print(f"Document: {result['page_content']}")


# COMMAND ----------

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [81]:
import openai
import numpy as np
from langchain.embeddings import OpenAIEmbeddings
from sklearn.cluster import KMeans
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# Configurar a chave da API do OpenAI
openai.api_key = 'YOUR_OPENAI_API_KEY'

# Função para gerar a descrição do cluster com ChatGPT-4 (via OpenAI)
def generate_cluster_description(cluster_embedding, model="gpt-4"):
    # Converter o embedding para um formato legível para o modelo
    prompt = f"Given the following vector representation of a group of documents, summarize the main category or theme of the documents represented by this vector: {cluster_embedding.tolist()}"
    
    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=50,  # Tamanho da resposta
        temperature=0.7  # Controlar a criatividade
    )
    
    return response.choices[0].text.strip()

# Função para gerar a média dos embeddings de documentos em cada cluster
def generate_cluster_embeddings(categorized_docs, document_embeddings):
    cluster_embeddings = {}
    for cluster_label, docs in categorized_docs.items():
        # Obter os embeddings dos documentos no cluster
        embeddings = [document_embeddings[doc['id'] - 1] for doc in docs]  # Índices começam em 1
        # Calcular a média dos embeddings para o cluster
        avg_embedding = np.mean(embeddings, axis=0)
        cluster_embeddings[cluster_label] = avg_embedding
    return cluster_embeddings

# Função para aplicar K-Means e categorizar os documentos
def kmeans_category_clustering(documents, embeddings, n_clusters=3):
    # Aplicando K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(embeddings)
    
    # Associar documentos aos seus clusters
    categorized_docs = {}
    for i, label in enumerate(kmeans.labels_):
        if label not in categorized_docs:
            categorized_docs[label] = []
        categorized_docs[label].append(documents[i])

    return categorized_docs, kmeans

# Exemplo de documentos (substitua com seus próprios documentos)
documents_dict = [
    {"id": 1, "page_content": "Recycled Waterhog Dog Mat for pets"},
    {"id": 2, "page_content": "Outdoor Dog Mat perfect for dogs"},
    {"id": 3, "page_content": "Women's Campside Oxfords, perfect for walking"},
    {"id": 4, "page_content": "Running Shoes for athletes"},
    {"id": 5, "page_content": "Infant and Toddler Girls' Coastal Chill Swimsuit"}
]

# Configurar o modelo de embeddings da OpenAI
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# Função para gerar embeddings para os documentos
def generate_embeddings(documents, embedding_model):
    embeddings = []
    for doc in documents:
        embedding = embedding_model.embed_documents(doc["page_content"])  # LLM Embedder
        embeddings.append(embedding)
    return np.array(embeddings)

# Gerar embeddings para os documentos
document_embeddings = generate_embeddings(documents_dict, embedding_model)

# Gerar os clusters com K-Means
categorized_docs, kmeans_model = kmeans_category_clustering(documents_dict, document_embeddings, n_clusters=3)

# Gerar embeddings para os clusters
cluster_embeddings = generate_cluster_embeddings(categorized_docs, document_embeddings)

# Usar ChatGPT-4 para gerar descrições de categoria para cada cluster
cluster_descriptions = {}

for cluster_label, cluster_embedding in cluster_embeddings.items():
    description = generate_cluster_description(cluster_embedding)
    cluster_descriptions[cluster_label] = description

# Exibir as descrições dos clusters
for cluster_label, description in cluster_descriptions.items():
    print(f"Cluster {cluster_label} Description: {description}")


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.